In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.6 MB/s 
     |████████████████████████████████| 895 kB 56.1 MB/s 
     |████████████████████████████████| 636 kB 62.2 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import classification_report
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW# get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import time

!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/utils.py .
from utils import *
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/Custom_Dataset_Class.py .
from Custom_Dataset_Class import CustomDataset
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/pytorchtools.py .
from pytorchtools import EarlyStopping
#from Bert_Classification import Bert_Classification_Model
#from RoBERT import RoBERT_Model

#from BERT_Hierarchical import BERT_Hierarchical_Model
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [4]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [6]:
%matplotlib inline

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [7]:
#change to where you store mimic3 data
MIMIC_3_DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/datasets'

train_df = pd.read_csv('%s/train_10.csv' % MIMIC_3_DIR)
eval_df = pd.read_csv('%s/dev_10.csv' % MIMIC_3_DIR)
test_df = pd.read_csv('%s/test_10.csv' % MIMIC_3_DIR)

train_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,rsbi,584.9;427.31,1
1,61638,103816,Nursing,title,414.01,1
2,61638,103816,General,title,414.01,1
3,23706,186321,Nursing/other,npn,401.9;428.0;530.81,1
4,55265,191108,General,title,530.81;584.9;427.31,1


In [8]:
full_df = pd.concat([train_df, eval_df, test_df], ignore_index=True)

In [9]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,rsbi,"[584.9, 427.31]",1
1,61638,103816,Nursing,title,[414.01],1
2,61638,103816,General,title,[414.01],1
3,23706,186321,Nursing/other,npn,"[401.9, 428.0, 530.81]",1
4,55265,191108,General,title,"[530.81, 584.9, 427.31]",1


In [10]:
#load multi label binarizer for one-hot encoding
mlb = MultiLabelBinarizer(sparse_output=True)

#labels_onehot = mlb.fit_transform(train_df.pop('LABELS'))
#labels_onehot[0][1]

In [11]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
0,17341,151110,Nursing/other,rsbi,1,0,0,0,0,1,0,0,0,1,0
1,61638,103816,Nursing,title,1,0,0,0,1,0,0,0,0,0,0
2,61638,103816,General,title,1,0,0,0,1,0,0,0,0,0,0
3,23706,186321,Nursing/other,npn,1,0,0,1,0,0,1,0,1,0,0
4,55265,191108,General,title,1,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,admission date discharge date date of birth se...,4644,0,0,1,0,0,0,1,0,0,0
294461,99650,199859,Discharge summary,admission date discharge date date of birth se...,5126,0,0,0,1,1,1,1,0,1,1
294462,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,1,1,0,0,0,0,0,0,0
294463,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,0,0,0,0,0,0,1,0,0,1


In [12]:
## Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

#full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()


,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
0,17341,151110,Nursing/other,rsbi,1,0,0,0,0,1,0,0,0,1,0
1,61638,103816,Nursing,title,1,0,0,0,1,0,0,0,0,0,0
2,61638,103816,General,title,1,0,0,0,1,0,0,0,0,0,0
3,23706,186321,Nursing/other,npn,1,0,0,1,0,0,1,0,1,0,0
4,55265,191108,General,title,1,0,0,0,0,1,0,0,1,1,0


In [13]:
full_df = full_df.drop(full_df[full_df['length']<300].index)

In [14]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
155040,65582,194117,Nursing,name8 md md h p yo male with strep bovis endoc...,300,0,0,0,0,0,0,1,0,0,1
155041,70924,119848,Nursing,42f with chronic alcoholic pancreatitis transf...,300,0,0,0,0,0,0,1,0,1,1
155042,25139,180782,Radiology,am picc line placment sch clip clip number rad...,300,0,0,0,0,0,0,0,0,1,0
155043,13494,164195,Radiology,pm ct abdomen w o contrast ct pelvis w o contr...,300,0,0,0,0,0,0,1,0,0,1
155044,30275,152118,Radiology,pm perc g g j tube plmt clip clip number radio...,300,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,admission date discharge date date of birth se...,4644,0,0,1,0,0,0,1,0,0,0
294461,99650,199859,Discharge summary,admission date discharge date date of birth se...,5126,0,0,0,1,1,1,1,0,1,1
294462,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,1,1,0,0,0,0,0,0,0
294463,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,0,0,0,0,0,0,1,0,0,1


In [15]:
train_df, test_df = train_test_split(full_df, test_size=0.2)

In [16]:
train_df, dev_df = train_test_split(train_df, test_size=0.2)

In [17]:
train_df.sort_values(['length'], inplace=True)
dev_df.sort_values(['length'], inplace=True)
test_df.sort_values(['length'], inplace=True)


In [18]:
train_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
0,43937,151838,Echo,patient test information indication coronary a...,300,0,0,0,1,0,1,0,0,1,0
1,59278,135559,Radiology,pm pelvis limited pelvis u s transvaginal clip...,300,0,0,0,0,0,0,1,0,0,0
2,11995,132213,Nursing/other,pmicu nursing progress 7a 7p review of systems...,300,0,0,0,0,0,0,0,1,0,0
3,6451,183196,Radiology,pm ct chest w contrast clip clip number radiol...,300,1,0,0,1,0,0,0,0,0,0
4,9719,159509,Echo,patient test information indication left ventr...,300,1,0,0,0,1,1,0,0,0,0


In [29]:
test_pred = test_df.copy()

In [36]:
X_train = train_df.TEXT
X_test = test_df.TEXT


NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in icd_classes_50:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train_df[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    test_pred[category] = prediction


... Processing 250.00
... Processing 272.4
... Processing 401.9
... Processing 414.01
... Processing 427.31
... Processing 428.0
... Processing 518.81
... Processing 530.81
... Processing 584.9
... Processing 599.0


In [37]:
test_df['labels'] = test_df[icd_classes_50].values.tolist()
test_pred['labels'] = test_pred[icd_classes_50].values.tolist()

In [38]:
outputs = np.vstack([test_pred['labels'][i] for i in test_pred.index])
targets = np.vstack([test_df['labels'][i] for i in test_df.index])

In [39]:
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.35439573257025553
F1 Score (Macro) = 0.2438386887258897


In [40]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.9375    0.0041    0.0081      3677
       272.4     0.4043    0.0051    0.0101      3714
       401.9     0.6429    0.3543    0.4568      7167
      414.01     0.6492    0.2286    0.3382      4120
      427.31     0.8407    0.1788    0.2949      5817
       428.0     0.7639    0.2101    0.3296      5773
      518.81     0.7288    0.6966    0.7123      6743
      530.81     0.6000    0.0012    0.0023      2567
       584.9     0.9723    0.1629    0.2791      5610
       599.0     0.8571    0.0035    0.0070      3421

   micro avg     0.7261    0.2344    0.3544     48609
   macro avg     0.7397    0.1845    0.2438     48609
weighted avg     0.7483    0.2344    0.3035     48609
 samples avg     0.4759    0.2520    0.3042     48609



In [41]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.5999590007838561
RUC AUC Score (Macro) = 0.5724213152445663
